# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [45]:
#Your code here
import pandas as pd
import scipy.stats as stats
import numpy as np

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [46]:
'''
H0: The click conversion of experimental group was significantly less than or equal to the click conversion of control
HA: The click conversion of experimental group was significantly more than the click conversion of the control
'''

'\nH0: The click conversion of experimental group was significantly less than or equal to the click conversion of control\nHA: The click conversion of experimental group was significantly more than the click conversion of the control\n'

In [47]:
#Read in Dataset
df = pd.read_csv('homepage_actions.csv')

In [48]:
# Check if there are ids that exist in both control and experiment
df[df['group'] == 'control'].merge(df[df['group'] == 'experiment'],on=['id'],how='inner')

,timestamp_x,id,group_x,action_x,timestamp_y,group_y,action_y


In [49]:
# # SORT BY TIMESTAMP
df.sort_values('timestamp')

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [50]:
# # VIEW COMES BEFORE THE CLICK
df.iloc[8180:8184]

,timestamp,id,group,action
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8181,2017-01-18 09:07:37.661143,795585,control,view
8182,2017-01-18 09:09:17.363917,795585,control,click
8183,2017-01-18 09:11:41.984113,192060,experiment,view


In [51]:
# # KEEP THE CLICKS AND DROP THE VIEWS WHERE THEY ALSO CLICKED
df.drop_duplicates(subset='id',keep='last',inplace=True)

In [52]:
df_control = df[df['group'] == 'control']
df_experiment = df[df['group'] == 'experiment']

In [53]:
# clicks_control    views_control
# clicks_exper      views_exper

table = np.array([[ df_control.value_counts('action')[1]  , df_control.value_counts('action')[0]],
                [df_experiment.value_counts('action')[1] , df_experiment.value_counts('action')[0]]])

In [55]:
chi2,p_value,dof,expected = stats.chi2_contingency(table)

p_value

0.00957168049704227

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [57]:
#Your code here
expected[1][0]

880.6194690265487

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [70]:
#Your code here

# standard dev of experiment = n * p * (1 - p) where p is click conversion rate and n is number of observations

std_experiment = (df_experiment.shape[0] * df_experiment.value_counts('action')[1]/df_experiment.shape[0] * (1-df_experiment.value_counts('action')[1]/df_experiment.shape[0])) ** 0.5

z = (df_experiment.value_counts('action')[1] - expected[1][0]) / std_experiment

z

1.8720677965010815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [71]:
#Your code here
stats.norm.sf(z)

0.03059861229979294

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

Yes - both p-values are less than the alpha (0.05)

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.